### Урок 3. ДЗ ###

1. *Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
X = np.array([ [   1,    1,  500,    1],
               [   1,    1,  700,    1],
               [   1,    2,  750,    2],
               [   1,    5,  600,    1],
               [   1,    3, 1450,    2],
               [   1,    0,  800,    1],
               [   1,    5, 1500,    3],
               [   1,   10, 2000,    3],
               [   1,    1,  450,    1],
               [   1,    2, 1000,    2]], dtype=np.float64)

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype=np.float64)

In [3]:
def standard_scale(x):
    res = (x - x.mean()) / x.std()
    return res

In [4]:
X_st = X.copy()
X_st[:, 2] = standard_scale(X[:, 2])

X_st

array([[ 1.        ,  1.        , -0.97958969,  1.        ],
       [ 1.        ,  1.        , -0.56713087,  1.        ],
       [ 1.        ,  2.        , -0.46401617,  2.        ],
       [ 1.        ,  5.        , -0.77336028,  1.        ],
       [ 1.        ,  3.        ,  0.97958969,  2.        ],
       [ 1.        ,  0.        , -0.36090146,  1.        ],
       [ 1.        ,  5.        ,  1.08270439,  3.        ],
       [ 1.        , 10.        ,  2.11385144,  3.        ],
       [ 1.        ,  1.        , -1.08270439,  1.        ],
       [ 1.        ,  2.        ,  0.05155735,  2.        ]])

In [42]:
# def calc_logloss(y, y_pred):
#     err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
#     return err


# доработанная функция
def calc_logloss(y, y_pred, eps=1e-15):
    err = 0
    for i in range(len(y)):
        y_pred[i] = max(eps, min(1 - eps, y_pred[i])) # добавление малой составл. если выходит 0
        err += y[i] * np.log(y_pred[i]) + (1.0 - y[i]) * np.log(1.0 - y_pred[i])
    err = - err / len(y)
    return err

In [41]:
# Плохой пример применения
y1 = np.array([1, 0])
y_pred1 = np.array([1, 0.2])
calc_logloss(y1, y_pred1)

0.11157177565710535

In [40]:
#ПРОВЕРКА

from sklearn.metrics import log_loss
log_loss(y1, y_pred1)

0.11157177565710535

<hr>

2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.


In [46]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [45]:
def eval_model(X, y, iterations, eta=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[1])
    n = X.shape[0]
    
    for i in range(iterations):
        z = np.dot(X, W)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        
        dQ = 1/n * X.T @ (y_pred - y)
        W -= eta * dQ
        if i % (iterations / 10) == 0:
            print(i, W, err)
    return W

In [89]:
W = eval_model(X_st, y, iterations=5000, eta=1e-3)

0 [ 0.49633477 -0.13971518  0.64766116  1.52246371] 1.1785958344356262
500 [ 0.36227303 -0.61296537  0.6357139   1.33157192] 0.5531388819143024
1000 [ 0.30073233 -0.70026449  0.66312517  1.28874272] 0.5201380455701675
1500 [ 0.25670353 -0.7153335   0.69622253  1.28005534] 0.5132905694561168
2000 [ 0.21863358 -0.7169718   0.72742369  1.28032147] 0.5084253284466037
2500 [ 0.18384751 -0.71636814  0.75565267  1.28410385] 0.5043772097533477
3000 [ 0.15151323 -0.71596113  0.78093944  1.28996596] 0.500934768374212
3500 [ 0.12124459 -0.71620567  0.80354008  1.29741256] 0.49796704155232696
4000 [ 0.09277701 -0.7171091   0.82373835  1.30618063] 0.4953727678019912
4500 [ 0.06589148 -0.7185895   0.84179994  1.31607405] 0.49307286149256413


In [79]:
# ОТВЕТ: считаю данные параметры оптимальными, т.к. при уменьшении eta (в данном случае)
# уменьшается log_loss, но растут веса, а увеличение количества итераций увеличивает время работы
# и приводит примерно к тем же результатам

<hr>

3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).


In [118]:
def calc_pred_proba(W, X):
    p = 1 / (1 + np.exp(-1*np.dot(X, W)))
    return p

In [112]:
calc_pred_proba(W, X_st)

array([0.45174437, 0.53997372, 0.70162611, 0.05220782, 0.79786736,
       0.7422546 , 0.793685  , 0.20248687, 0.42994129, 0.78539386])

In [110]:
y

array([0., 0., 1., 0., 1., 0., 1., 0., 1., 1.])

<hr>

4. Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [121]:
def calc_pred(W, X, p=.5):
    y_pred = np.array([1 if i >= p else 0 for i in calc_pred_proba(W, X)])
    return y_pred

In [192]:
y_pred = calc_pred(W, X_st)
y_pred

array([0, 1, 1, 0, 1, 1, 1, 0, 0, 1])

<hr>

5. *Реализуйте функции для подсчета Accuracy, матрицы ошибок, точности и полноты, а также F1 score.

In [177]:
N = true_values.shape[1]
accuracy = (true_values == predictions).sum() / N
TP = ((predictions == 1) & (true_values == 1)).sum()
FP = ((predictions == 1) & (true_values == 0)).sum()
precision = TP / (TP+FP)

False

In [180]:
(y == y_pred).sum() 

7

In [204]:
def get_pred_param(y, y_pred):
    n = y.shape[0]
    accuracy = (y == y_pred).sum() / n
    TP = ((y_pred == 1) & (y == 1)).sum()
    FP = ((y_pred == 1) & (y == 0)).sum()
    FN = ((y_pred == 0) & (y == 1)).sum()
    TN = ((y_pred == 0) & (y == 10)).sum()
    
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    F = 2 * precision * recall / (precision + recall)
    
    errors_matrix = np.zeros([2, 2])
    errors_matrix[0][0] = TP
    errors_matrix[0][1] = FP
    errors_matrix[1][0] = FN
    errors_matrix[1][1] = 
    
    print(f'Accuracy: {accuracy:0.2f}')
    print(f'Precision: {precision:0.2f}')
    print(f'Recall: {recall:0.2f}')
    print(f'F: {F:0.2f}') 
    print(f'Errors matrix: \n {errors_matrix}')
    
    return None

In [205]:
get_pred_param(y, y_pred)

Accuracy: 0.70
Precision: 0.67
Recall: 0.80
F: 0.73
Errors matrix: 
 [[4. 2.]
 [0. 0.]]
